In [1]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score, auc
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix,classification_report,accuracy_score,precision_score,recall_score,f1_score,roc_auc_score
from sklearn.metrics import f1_score
from sklearn.metrics import hamming_loss
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import accuracy_score
from scipy import interp

from sklearn import preprocessing
from sklearn.preprocessing import MultiLabelBinarizer

In [2]:
import tensorflow as tf

from keras.models import Sequential
from keras.layers import *
from keras.optimizers import *
from keras.losses import binary_crossentropy
from keras.metrics import *
from keras import callbacks
from keras.callbacks import EarlyStopping
from sklearn.metrics import roc_auc_score, auc, precision_recall_curve, confusion_matrix
import numpy as np
import sklearn.metrics as metrics
from collections import Counter
import pandas as pd
import random
from sklearn.model_selection import KFold
from tqdm import tqdm
import scipy.sparse as sp
from copy import deepcopy
import warnings 
import os
from sklearn import preprocessing

from sklearn.preprocessing import MultiLabelBinarizer

C:\ProgramData\Anaconda3\envs\GCNFTG\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\ProgramData\Anaconda3\envs\GCNFTG\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\ProgramData\Anaconda3\envs\GCNFTG\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\ProgramData\Anaconda3\envs\GCNFTG\lib\site-packages\tensorflow\pyth

In [3]:
def load_data(directory):
    D_SSM = np.loadtxt(directory + '/D_SM归一化.txt')
    M_FSM = np.loadtxt(directory + '/M_SM归一化.txt')

    #D_SSM = np.loadtxt(directory + '/D_SM.txt')
    #M_FSM = np.loadtxt(directory + '/M_SM.txt')



    ID = np.zeros(shape=(D_SSM.shape[0], D_SSM.shape[1]))
    IM = np.zeros(shape=(M_FSM.shape[0], M_FSM.shape[1]))
    for i in range(D_SSM.shape[0]):
        for j in range(D_SSM.shape[1]):
            if D_SSM[i][j] == 0:
                ID[i][j] = D_SSM[i][j]######D_GSM[i][j]
            else:
                ID[i][j] = D_SSM[i][j]
    for i in range(M_FSM.shape[0]):
        for j in range(M_FSM.shape[1]):
            if M_FSM[i][j] == 0:
                IM[i][j] = M_FSM[i][j]#######M_GSM[i][j]
            else:
                IM[i][j] = M_FSM[i][j]
                
    ID = pd.DataFrame(ID).reset_index()
    IM = pd.DataFrame(IM).reset_index()
    ID.rename(columns = {'index':'id'}, inplace = True)
    IM.rename(columns = {'index':'id'}, inplace = True)
    ID['id'] = ID['id'] + 1
    IM['id'] = IM['id'] + 1
    
    return ID, IM

def sample(directory, random_seed):
    all_associations = pd.read_csv(directory + '/drug_mutation_pairs.csv', names=['Drug', 'Mutation', 'label'])
    known_associations = all_associations.loc[all_associations['label'] == 1]
    known_associations_resistance=all_associations.loc[all_associations['label'] == -1]
    unknown_associations = all_associations.loc[all_associations['label'] == 0]
    #print('known_associations',known_associations)
    #print('known_associations_resistance',known_associations_resistance)
    
    #random_negative = unknown_associations.sample(n=known_associations.shape[0], random_state=random_seed, axis=0)
    random_negative = unknown_associations.sample(n=known_associations.shape[0]+known_associations_resistance.shape[0], random_state=random_seed, axis=0)
    #print('all_associations',len(known_associations))
    #print('random_negative',random_negative)
    #sample_df = known_associations.append(random_negative)
    sample_df = known_associations.append(known_associations_resistance.append(random_negative))
    print('sample_df',sample_df)
    #np.savetxt("C:/Users/Administrator/Desktop/图采样有向图代码/test data/参数/sample_df.txt", sample_df,fmt='%s', newline='\n')
    sample_df.reset_index(drop=True, inplace=True)
   
    return sample_df

In [4]:
def performances(class_arr,y_true, y_pred, y_prob):

    #y_true = np.array(y_true)
    ##随机森林
    #print('y_true',y_true)
    #print('y_pred',y_pred)   
    #print('y_prob',y_prob)
    
    #y_prob_prob = y_prob.reshape(-1)
    #print('y_prob_prob',y_prob_prob)
    
    #####fpr, tpr, _ = roc_curve(y_true, y_prob,pos_label=1)
    #fpr, tpr, _ = roc_curve(y_true, y_prob)
    #####roc_auc = auc(fpr, tpr)
    #print('fpr',fpr)
    #print('tpr',tpr)
    #print('roc_auc',roc_auc)
   

    
    #roc_auc = roc_auc_score(y_true, y_pred, average='macro')
    #print('roc_auc',roc_auc)


    #####pr, re, _ = precision_recall_curve(y_true, y_prob,pos_label=1)
    #pr, re, _ = precision_recall_curve(y_true, y_prob)
    #print('pr',pr)
    #print('re',re)
    #####aupr = auc(re, pr)
    #print('aupr',aupr)
    #y_true = np.argmax(y_true, axis=1)
    #y_pred = np.argmax(y_pred, axis=1) 
    
    y_true_prob=class_arr.reshape(-1)
    y_pred_prob = y_prob.reshape(-1)
    #y_true_prob = np.argmax(class_arr, axis=1)
    #y_pred_prob = np.argmax(y_prob, axis=1) 
    
    #print('y_true_prob',y_true_prob)
    #print('y_pred_prob',y_pred_prob)
    fpr, tpr, _ = roc_curve(y_true_prob, y_pred_prob)
    roc_auc = auc(fpr, tpr)
    
    pr, re, _ = precision_recall_curve(y_true_prob, y_pred_prob)
    aupr = auc(re, pr)
    
    accuracy=accuracy_score(y_true, y_pred)
    f1=f1_score(y_true, y_pred, average="macro")
    recall=recall_score(y_true, y_pred, average='macro')
    precision=precision_score(y_true, y_pred, average='macro')
    
    
    
    
    
    #tn, fp, fn, tp = confusion_matrix(y_true, y_pred, labels = [0, 1]).ravel().tolist()

    #pos_acc = tp / sum(y_true)
    #neg_acc = tn / (len(y_pred) - sum(y_pred)) # [y_true=0 & y_pred=0] / y_pred=0
    #accuracy = (tp+tn)/(tn+fp+fn+tp)
    
    #recall = tp / (tp+fn)
    #precision = tp / (tp+fp)
    #f1 = 2*precision*recall / (precision+recall)
    
    #roc_auc = roc_auc_score(y_true, y_prob)
    #prec, reca, _ = precision_recall_curve(y_true, y_prob)
    #aupr = auc(reca, prec)
    
    #print('tn = {}, fp = {}, fn = {}, tp = {}'.format(tn, fp, fn, tp))
    #print('y_pred: 0 = {} | 1 = {}'.format(Counter(y_pred)[0], Counter(y_pred)[1]))
    #print('y_true: 0 = {} | 1 = {}'.format(Counter(y_true)[0], Counter(y_true)[1]))
    #print('acc={:.4f}|precision={:.4f}|recall={:.4f}|f1={:.4f}|auc={:.4f}|aupr={:.4f}|pos_acc={:.4f}|neg_acc={:.4f}'.format(accuracy, precision, recall, f1, roc_auc, aupr, pos_acc, neg_acc))
    print('acc={:.4f}|precision={:.4f}|recall={:.4f}|f1={:.4f}|auc={:.4f}|aupr={:.4f}'.format(accuracy, precision, recall, f1, roc_auc,aupr))
    return (y_true, y_pred, y_prob), (accuracy, precision, recall, f1, roc_auc, aupr)

In [5]:
def obtain_data(directory, isbalance):

    ID, IM = load_data(directory)

    if isbalance:
        dtp = sample(directory, random_seed = 1234)
    else:
        dtp = pd.read_csv(directory + '/drug_mutation_pairs.csv', names=['Drug', 'Mutation', 'label'])

    mirna_ids = list(set(dtp['Drug']))
    disease_ids = list(set(dtp['Mutation']))
    random.shuffle(mirna_ids)
    random.shuffle(disease_ids)
    print('# Drug = {} | Mutation = {}'.format(len(mirna_ids), len(disease_ids)))

    mirna_test_num = int(len(mirna_ids) / 5)
    disease_test_num = int(len(disease_ids) / 5)
    print('# Test: Drug = {} | Mutation = {}'.format(mirna_test_num, disease_test_num))    
    
    samples = pd.merge(pd.merge(dtp, ID, left_on = 'Drug', right_on = 'id'), IM, left_on = 'Mutation', right_on = 'id')
    #print('samples',samples)
    samples.drop(labels = ['id_x', 'id_y'], axis = 1, inplace = True)
    #print('samples',samples)
    #np.savetxt("C:/Users/Administrator/Desktop/图采样有向图代码/test data/参数/samples.txt", samples,fmt='%s', newline='\n')
    return ID, IM, dtp, mirna_ids, disease_ids, mirna_test_num, disease_test_num, samples

In [6]:
def generate_task_Tp_train_test_idx(samples):
    kf = KFold(n_splits = 5, shuffle = True, random_state = 1234)

    train_index_all, test_index_all, n = [], [], 0
    train_id_all, test_id_all = [], []
    fold = 0
    for train_idx, test_idx in tqdm(kf.split(samples.iloc[:, 3:])): #train_index与test_index为下标
        print('-------Fold ', fold)
        train_index_all.append(train_idx) 
        test_index_all.append(test_idx)

        train_id_all.append(np.array(dtp.iloc[train_idx][['Drug', 'Mutation', 'label']]))
        test_id_all.append(np.array(dtp.iloc[test_idx][['Drug', 'Mutation', 'label']]))
        #print('train_index_all',train_index_all)
        #print('test_index_all',test_index_all)

        print('# Pairs: Train = {} | Test = {}'.format(len(train_idx), len(test_idx)))
        fold += 1
    #print('train_id_all',train_id_all)   
    return train_index_all, test_index_all, train_id_all, test_id_all

In [15]:
def generate_task_Tm_Td_train_test_idx(item, ids, dtp):
    
    test_num = int(len(ids) / 5)
    
    train_index_all, test_index_all = [], []
    train_id_all, test_id_all = [], []
    
    for fold in range(5):
        print('-------Fold ', fold)
        if fold != 4:
            test_ids = ids[fold * test_num : (fold + 1) * test_num]
        else:
            test_ids = ids[fold * test_num :]

        train_ids = list(set(ids) ^ set(test_ids))
        print('# {}: Train = {} | Test = {}'.format(item, len(train_ids), len(test_ids)))

        test_idx = dtp[dtp[item].isin(test_ids)].index.tolist()
        train_idx = dtp[dtp[item].isin(train_ids)].index.tolist()
        random.shuffle(test_idx)
        random.shuffle(train_idx)
        print('# Pairs: Train = {} | Test = {}'.format(len(train_idx), len(test_idx)))
        assert len(train_idx) + len(test_idx) == len(dtp)

        train_index_all.append(train_idx) 
        test_index_all.append(test_idx)
        
        train_id_all.append(train_ids)
        test_id_all.append(test_ids)
        #print('test_id_all.append(test_ids)',test_id_all.append(test_ids))
        
    return train_index_all, test_index_all, train_id_all, test_id_all

# 随机森林

In [30]:
def run_rf(train_index_all, test_index_all, samples):
    
    fold = 0
    for train_idx, test_idx in zip(train_index_all, test_index_all):
        print('-----------------------Fold = ', str(fold))

        X = samples.iloc[:, 3:]
        y = samples['label']
        #print('X',X)
        #print('y',y)
        

        
        
        
        #np.savetxt("C:/Users/Administrator/Desktop/图采样有向图代码/test data/参数/y.txt", y,fmt='%s', newline='\n')
        
        scaler = preprocessing.MinMaxScaler().fit(X.iloc[train_idx,:])
        X = scaler.transform(X)
        #print('y[train_idx]',y[train_idx])
        print('X',X.shape)

        x_train, y_train = X[train_idx], y[train_idx]
        x_test, y_test = X[test_idx], y[test_idx]
        
        #print('x_train',x_train)
        #print('x_train.shape',x_train.shape)
        print('x_test',x_test)
        print('x_test.shape',x_test.shape)
        #np.savetxt("C:/Users/Administrator/Desktop/图采样有向图代码/test data/分类散点图数据/GCN/ys_test[2].txt", x_test,fmt='%s', newline='\n')
        #print('y_train',y_train)
        #print('y_train.shape',y_train.shape)
        #print('y_test',y_test)
        #print('y_test.shape',y_test.shape)
        #s=pd.Series(y_train)
        #pd.DataFrame(y_train).to_csv("F:/graph data/train_num/train_num.csv")
        #pd.DataFrame(y_test).to_csv("F:/graph data/train_num/test_num.csv")

        #clf = RandomForestClassifier(n_estimators=100, random_state = 19961231)
        clf = RandomForestClassifier(random_state =19961231)
        clf.fit(x_train, y_train)###从训练集中构建一个树木森林
        
        #yy = np.array(y_train.shape[0])##2806
        df=pd.DataFrame(y_train, columns=['label'])
        b=df['label'].tolist()
        NewList = [[x] for x in b]
        cf = MultiLabelBinarizer()
        class_arr=cf.fit_transform(NewList)
        print('class_arr',class_arr)
        #print("The new lists of lists: ",NewList)
         #print('df',type(df))
        #class_map=dict(df)
        #print('ddf',class_map) 
        #a=df.index.tolist()
        #print('class_arr',class_arr)
        #print(class_arr.fit_transform(NewList) )
        
        ###test  标签
        
        df_test=pd.DataFrame(y_test, columns=['label'])
        b_test=df_test['label'].tolist()
        NewList_test = [[x] for x in b_test]
        cf_test = MultiLabelBinarizer()
        class_arr_test=cf.fit_transform(NewList_test)
        
        print('df_test',df_test)
        print('df_test',df_test.shape)
        #np.savetxt("C:/Users/Administrator/Desktop/图采样有向图代码/test data/分类散点图数据/GCN/ys_test[0]g.txt", df_test,fmt='%s', newline='\n')
        #print('cf_test',cf_test)
        #print('cf_test',cf_test.shape)
    
        #df = pd.DataFrame(data=y_train, columns=['Col1'])
        #class_map = list( df["label"].items())             ###转换成list格式
        #print('class_map',class_map)
        
        #a_array = np.array(class_map)   ##数组
        #print('a_array',a_array)
        #class_map=dict(a_array)
        #print('dict(class_map)',dict(class_map))
        #class_map1 = {int(k):v for k,v in class_map.items()}
        #print('class_map1',class_map1)
        #assert len(class_map1) == y_train.shape[0]
        
        #print('b',b)
        #a_array = np.array(class_map)
        #b_array = np.array(b)
        #print('a_array',a_array)
        #print('b_array',b_array)        
        
        
        ###多分类标签转化成01表示，但是不适用我们的数据
        #mlb = preprocessing.MultiLabelBinarizer()
        #cf=mlb.fit_transform(class_map)
        #print('1',cf)
        #####全部需要#### 
        #num_vertices = y_train.shape[0]
        
        #if isinstance(list(class_map1.values())[0],list):
        #    num_classes = len(list(class_map1.values())[0]) 
        #    class_arr = np.zeros((num_vertices, num_classes))
        #    for k,v in class_map1.items():
        #        class_arr[k] = v 
        #else:
            
        #   num_classes=max(class_map1.values()) - min(class_map1.values()) + 1
        #   #print('num_classes',num_classes)
        #    offset=min(class_map1.values())
            #print('offset',offset)
        #    class_arr = np.zeros((2806, num_classes))
            #print('class_arr',class_arr.shape)
            #print('class_map.items()',class_map.items())
        #    cf=class_map1.items()
        #    print('cf',type(cf))
        #    for k,v in class_map1.items():
        #        class_arr[k][v-offset]=1
        #print('class_arr',class_arr)
        #print('class_arr',class_arr.shape)
        
        
        
        
        
        
        #print('yy',yy)
        #class_map = dict()
        #for i  in range(3508):
        #    class_map[str(i)] = y[i].tolist()

        #print('class_map:',class_map)
    
        #class_map_1 = {int(1):v for k,v in class_map.items()}
        #class_map = {int(k):v for k,v in class_map.items()}
        
        #df=np.array(df)
        #num_vertices = y_train.shape[0]
        #print('num_vertices',num_vertices)
        #num_classes = max(class_map.values()) - min(class_map.values()) + 1
        #num_classes=3
        #offset=min(class_map.values())
        #class_arr = np.zeros((num_vertices, num_classes))
        #print('class_arr',class_arr)
        
        #print('class_map.items',class_map.items())
        #for k,v in class_map.items():
        #    class_arr[k][v-offset]=1
        #    [k][v-offset]=1
        #print('class_arr',class_arr)
        
        y_train_prob = clf.predict_proba(x_train)
        y_test_prob = clf.predict_proba(x_test)
        #print('x_train',x_train)
        #print('y_train_prob',y_train_prob)
        
        y_train_pred = clf.predict(x_train)
        y_test_pred = clf.predict(x_test)
        print('y_train_prob',y_train_prob)
        print('y_train_prob',y_train_prob.shape)
        print('y_test_prob',y_test_prob)
        print('y_test_prob',y_test_prob.shape)
        #print('y_train_pred',y_train_pred)
        #print('y_test_pred',y_test_pred)

        #print('y_train',y_train)
        print('Train:')
        ys_train, metrics_train = performances(class_arr,y_train, y_train_pred, y_train_prob)
        #ys_train, metrics_train = performances(y_train, y_train_pred, y_train_prob[:, 1])
        #ys_train, metrics_train = performances(y_train, y_train_pred, y_train_prob[:, 0])
        
        print('Test:')
        ys_test, metrics_test = performances(class_arr_test,y_test, y_test_pred, y_test_prob)
        #ys_test, metrics_test = performances(y_test, y_test_pred, y_test_prob[:, 1])
        #ys_test, metrics_test = performances(y_test, y_test_pred, y_test_prob[:, 0])

        fold += 1
    
    return ys_train, metrics_train, ys_test, metrics_test

## RF

In [31]:
#directory = 'C:/Users/Administrator/Desktop/MDA-GCNFTG-main/data'
directory='C:/Users/Administrator/Desktop/图采样data/last data'
#for isbalance in [False]:
for isbalance in [True]:
#for isbalance in [True, False]:
    
    ID, IM, dtp, mirna_ids, disease_ids, mirna_test_num, disease_test_num, samples = obtain_data(directory, isbalance)
    for task in ['Tp', 'Tm', 'Td']:
    #for task in ['Tp']:
        
        print('========== isbalance = {} | task = {}'.format(isbalance, task))
        
        if task == 'Tp':
            train_index_all, test_index_all, train_id_all, test_id_all = generate_task_Tp_train_test_idx(samples)
            
        elif task == 'Tm':
            item = 'Drug'
            ids = mirna_ids
            train_index_all, test_index_all, train_id_all, test_id_all = generate_task_Tm_Td_train_test_idx(item, ids, dtp)

        elif task == 'Td':
            item = 'Mutation'
            ids = disease_ids
            train_index_all, test_index_all, train_id_all, test_id_all = generate_task_Tm_Td_train_test_idx(item, ids, dtp)

        ys_train, metrics_train, ys_test, metrics_test = run_rf(train_index_all, test_index_all, samples)

5it [00:00, 385.64it/s]

sample_df         Drug  Mutation  label
153        1       154      1
1188       2       528      1
1772       3       451      1
2920       5       277      1
3207       5       564      1
...      ...       ...    ...
4878       8       252      0
20521     32        31      0
118974   180       656      0
12639     20        81      0
66290    101       191      0

[3508 rows x 3 columns]
# Drug = 184 | Mutation = 661
# Test: Drug = 36 | Mutation = 132
========== isbalance = True | task = Tp
-------Fold  0
# Pairs: Train = 2806 | Test = 702
-------Fold  1
# Pairs: Train = 2806 | Test = 702
-------Fold  2
# Pairs: Train = 2806 | Test = 702
-------Fold  3
# Pairs: Train = 2807 | Test = 701
-------Fold  4
# Pairs: Train = 2807 | Test = 701
-----------------------Fold =  0
X (3508, 845)
x_test [[0.         0.89851304 0.97045899 ... 0.560056   0.69342503 0.65846584]
 [0.95066492 0.99188569 0.99521619 ... 0.560056   0.69342503 0.65846584]
 [0.99368461 0.90122084 0.964258   ... 0.560056   

class_arr [[0 0 1]
 [0 0 1]
 [0 0 1]
 ...
 [0 1 0]
 [1 0 0]
 [1 0 0]]
df_test       label
0         1
1         1
2         1
11        1
13        1
...     ...
3472     -1
3480      0
3494      1
3498     -1
3505      0

[702 rows x 1 columns]
df_test (702, 1)
y_train_prob [[0.02 0.08 0.9 ]
 [0.01 0.15 0.84]
 [0.03 0.2  0.77]
 ...
 [0.07 0.92 0.01]
 [0.88 0.08 0.04]
 [0.78 0.19 0.03]]
y_train_prob (2806, 3)
y_test_prob [[0.03 0.38 0.59]
 [0.32 0.11 0.57]
 [0.08 0.13 0.79]
 ...
 [0.07 0.38 0.55]
 [0.51 0.23 0.26]
 [0.04 0.91 0.05]]
y_test_prob (702, 3)
Train:
acc=1.0000|precision=1.0000|recall=1.0000|f1=1.0000|auc=1.0000|aupr=1.0000
Test:
acc=0.6667|precision=0.6180|recall=0.6183|f1=0.6133|auc=0.8305|aupr=0.7190
-----------------------Fold =  1
X (3508, 845)
x_test [[0.98052622 0.96020941 0.99174189 ... 0.560056   0.69342503 0.65846584]
 [0.94001478 0.80196109 0.87985237 ... 0.560056   0.69342503 0.65846584]
 [0.99061974 0.8594114  0.94167571 ... 0.560056   0.69342503 0.65846584]
 ...

y_train_prob [[0.96 0.02 0.02]
 [0.9  0.08 0.02]
 [0.   0.99 0.01]
 ...
 [0.04 0.8  0.16]
 [0.16 0.04 0.8 ]
 [0.82 0.08 0.1 ]]
y_train_prob (2886, 3)
y_test_prob [[0.27 0.57 0.16]
 [0.05 0.66 0.29]
 [0.06 0.8  0.14]
 ...
 [0.84 0.12 0.04]
 [0.07 0.49 0.44]
 [0.14 0.78 0.08]]
y_test_prob (622, 3)
Train:
acc=1.0000|precision=1.0000|recall=1.0000|f1=1.0000|auc=1.0000|aupr=1.0000
Test:
acc=0.6254|precision=0.5715|recall=0.5475|f1=0.5483|auc=0.7971|aupr=0.6557
-----------------------Fold =  2
X (3508, 845)
x_test [[0.95045142 0.79196069 0.88206627 ... 0.0230039  0.8360868  0.85668815]
 [0.96700455 0.90016227 0.95035791 ... 0.98050724 0.02816016 0.02971166]
 [0.98706487 0.93605081 0.98462048 ... 0.90941104 0.00925557 0.02129169]
 ...
 [0.99299618 0.87970684 0.96137128 ... 0.95748515 0.20887417 0.18663875]
 [0.95982487 0.79900804 0.89368417 ... 0.07277218 0.74070534 0.76871312]
 [0.99224646 0.93913923 0.98992068 ... 0.86999729 0.34817062 0.31720169]]
x_test.shape (654, 845)
class_arr [[1 0 0]

y_train_prob [[0.32 0.63 0.05]
 [0.83 0.06 0.11]
 [0.86 0.14 0.  ]
 ...
 [0.06 0.27 0.67]
 [0.92 0.07 0.01]
 [0.01 0.76 0.23]]
y_train_prob (2776, 3)
y_test_prob [[0.07 0.78 0.15]
 [0.22 0.03 0.75]
 [0.12 0.02 0.86]
 ...
 [0.16 0.51 0.33]
 [0.69 0.3  0.01]
 [0.3  0.65 0.05]]
y_test_prob (732, 3)
Train:
acc=1.0000|precision=1.0000|recall=1.0000|f1=1.0000|auc=1.0000|aupr=1.0000
Test:
acc=0.6339|precision=0.5717|recall=0.5824|f1=0.5707|auc=0.7989|aupr=0.6724
-----------------------Fold =  3
X (3508, 845)
x_test [[8.45818988e-01 6.62288565e-01 7.61479100e-01 ... 4.96810822e-01
  7.48061116e-01 7.36228172e-01]
 [9.73958879e-01 9.68018267e-01 9.92913422e-01 ... 8.29432000e-04
  9.21116435e-01 9.32177887e-01]
 [9.93083022e-01 8.62788554e-01 9.44815515e-01 ... 2.48168100e-03
  9.24963978e-01 9.40942286e-01]
 ...
 [9.68832511e-01 8.22880216e-01 9.07407611e-01 ... 2.69222282e-01
  6.68373253e-01 7.18109976e-01]
 [8.68068386e-01 6.77510987e-01 7.77021061e-01 ... 9.91580353e-01
  1.22504082e-01 1.

# DNN

In [ ]:
def precision(y_true, y_pred):
    # Calculates the precision
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision


def recall(y_true, y_pred):
    # Calculates the recall
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def fbeta_score(y_true, y_pred, beta=1):
    # Calculates the F score, the weighted harmonic mean of precision and recall.

    if beta < 0:
        raise ValueError('The lowest choosable beta is zero (only precision).')
        
    # If there are no true positives, fix the F score at 0 like sklearn.
    if K.sum(K.round(K.clip(y_true, 0, 1))) == 0:
        return 0

    p = precision(y_true, y_pred)
    r = recall(y_true, y_pred)
    bb = beta ** 2
    fbeta_score = (1 + bb) * (p * r) / (bb * p + r + K.epsilon())
    return fbeta_score

def f1(y_true, y_pred):
    # Calculates the f-measure, the harmonic mean of precision and recall.
    return fbeta_score(y_true, y_pred, beta=1)


def transfer(y_pred):
    return [[0,1][x>0.5] for x in y_pred.reshape(-1)]

In [607]:
def Model(x):
    model = Sequential()
    model.add(Dense(1024, activation='elu', input_shape=(x.shape[1],)))
    model.add(Dense(512, activation='elu'))
    model.add(Dense(256, activation='relu'))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    optimizer = Adam(lr = 0.0001)
    model.compile(optimizer = optimizer, loss = binary_crossentropy, metrics=[binary_accuracy, f1, recall, precision])
    return model

In [608]:
def run_dnn(train_index_all, test_index_all, samples):
    
    fold = 0
    for train_idx, test_idx in zip(train_index_all, test_index_all):
        print('----------------------- Fold = ', str(fold))

        X = samples.iloc[:, 3:]
        y = samples['label']

        scaler = preprocessing.MinMaxScaler().fit(X.iloc[train_idx,:])
        X = scaler.transform(X)

        x_train, y_train = X[train_idx], y[train_idx]
        x_test, y_test = X[test_idx], y[test_idx]

        model = Model(x_train)
        early_stopping = EarlyStopping(monitor='val_loss', patience = 50)
        #model.fit(x_train, y_train, epochs =500, batch_size = 256, validation_data=(x_test, y_test), callbacks=[early_stopping],verbose = 2)
        model.fit(x_train, y_train, epochs =100, batch_size = 512, validation_data=(x_test, y_test), callbacks=[early_stopping],verbose = 2)

        y_train_pred, y_test_pred = model.predict(x_train, verbose = 0), model.predict(x_test, verbose = 0)
        y_train_prob, y_test_prob = model.predict_proba(x_train), model.predict_proba(x_test)

        if len(Counter(y_train_pred.reshape(-1))) > 2: 
            y_train_pred = transfer(y_train_pred)
        else:
            print(Counter(y_train_pred.reshape(-1)))
        if len(Counter(y_test_pred.reshape(-1))) > 2: 
            y_test_pred = transfer(y_test_pred)
        else:
            print(Counter(y_test_pred.reshape(-1)))

        performances_train = performances(y_train, y_train_pred, y_train_prob)
        performances_test = performances(y_test, y_test_pred, y_test_prob)
        #print('y_train',y_train)
        #print('y_train_pred',y_train_pred)
        #print('y_train_prob',y_train_prob)
        
        
        fold += 1
    
    return y_train_pred, y_test_pred, y_train_prob, y_test_prob, performances_train, performances_test

# Run

## DNN 

In [610]:
#directory = 'C:/Users/Administrator/Desktop/MDA-GCNFTG-main/data'
directory='C:/Users/Administrator/Desktop/图采样data/last data'
for isbalance in [True]:
#for isbalance in [True, False]:
    
    ID, IM, dtp, mirna_ids, disease_ids, mirna_test_num, disease_test_num, samples = obtain_data(directory, 
                                                                                                 isbalance)
    #for task in ['Tp', 'Tm', 'Td']:
    for task in ['Tp']:   
        print('========== isbalance = {} | task = {}'.format(isbalance, task))
        
        if task == 'Tp':
            train_index_all, test_index_all, train_id_all, test_id_all = generate_task_Tp_train_test_idx(samples)
            
        elif task == 'Tm':
            item = 'miRNA'
            ids = mirna_ids
            train_index_all, test_index_all, train_id_all, test_id_all = generate_task_Tm_Td_train_test_idx(item, ids, dtp)

        elif task == 'Td':
            item = 'disease'
            ids = disease_ids
            train_index_all, test_index_all, train_id_all, test_id_all = generate_task_Tm_Td_train_test_idx(item, ids, dtp)

        y_train_pred, y_test_pred, y_train_prob, y_test_prob, performances_train, performances_test = run_dnn(train_index_all, test_index_all, samples)

5it [00:00, 418.76it/s]

# Drug = 184 | Mutation = 661
# Test: Drug = 36 | Mutation = 132
========== isbalance = True | task = Tp
-------Fold  0
# Pairs: Train = 2806 | Test = 702
-------Fold  1
# Pairs: Train = 2806 | Test = 702
-------Fold  2
# Pairs: Train = 2806 | Test = 702
-------Fold  3
# Pairs: Train = 2807 | Test = 701
-------Fold  4
# Pairs: Train = 2807 | Test = 701
----------------------- Fold =  0


Train on 2806 samples, validate on 702 samples
Epoch 1/100
 - 3s - loss: 0.1217 - binary_accuracy: 0.4665 - f1: 0.0598 - recall: 0.1032 - precision: 0.0421 - val_loss: -5.5562e-02 - val_binary_accuracy: 0.5271 - val_f1: 0.0000e+00 - val_recall: 0.0000e+00 - val_precision: 0.0000e+00
Epoch 2/100
 - 0s - loss: -2.5212e-01 - binary_accuracy: 0.4932 - f1: 0.0000e+00 - recall: 0.0000e+00 - precision: 0.0000e+00 - val_loss: -1.0705e-01 - val_binary_accuracy: 0.5271 - val_f1: 0.0000e+00 - val_recall: 0.0000e+00 - val_precision: 0.0000e+00
Epoch 3/100
 - 0s - loss: -4.1455e-01 - binary_accuracy: 0.4932 - f1: 0.0000e+00 - recall: 0.0000e+00 - precision: 0.0000e+00 - val_loss: -1.3480e-01 - val_binary_accuracy: 0.5271 - val_f1: 0.0000e+00 - val_recall: 0.0000e+00 - val_precision: 0.0000e+00
Epoch 4/100
 - 0s - loss: -4.7181e-01 - binary_accuracy: 0.4932 - f1: 0.0000e+00 - recall: 0.0000e+00 - precision: 0.0000e+00 - val_loss: -1.0109e-01 - val_binary_accuracy: 0.5271 - val_f1: 0.0000e+00 - val_r

Epoch 34/100
 - 0s - loss: -5.9863e-01 - binary_accuracy: 0.4929 - f1: 0.0137 - recall: 0.0071 - precision: 0.3784 - val_loss: -3.6815e-01 - val_binary_accuracy: 0.5185 - val_f1: 0.0113 - val_recall: 0.0062 - val_precision: 0.0677
Epoch 35/100
 - 0s - loss: -5.9322e-01 - binary_accuracy: 0.4914 - f1: 0.0086 - recall: 0.0044 - precision: 0.1521 - val_loss: -3.5748e-01 - val_binary_accuracy: 0.5214 - val_f1: 0.0343 - val_recall: 0.0183 - val_precision: 0.2726
Epoch 36/100
 - 0s - loss: -6.1011e-01 - binary_accuracy: 0.4907 - f1: 0.0335 - recall: 0.0176 - precision: 0.3932 - val_loss: -3.9172e-01 - val_binary_accuracy: 0.5228 - val_f1: 0.0347 - val_recall: 0.0183 - val_precision: 0.3333
Epoch 37/100
 - 0s - loss: -6.2125e-01 - binary_accuracy: 0.4918 - f1: 0.0121 - recall: 0.0063 - precision: 0.1825 - val_loss: -3.4458e-01 - val_binary_accuracy: 0.5256 - val_f1: 0.0118 - val_recall: 0.0062 - val_precision: 0.1353
Epoch 38/100
 - 0s - loss: -5.8164e-01 - binary_accuracy: 0.4929 - f1: 0.003

Epoch 67/100
 - 0s - loss: -7.0778e-01 - binary_accuracy: 0.4939 - f1: 0.0087 - recall: 0.0044 - precision: 0.4562 - val_loss: -4.5907e-01 - val_binary_accuracy: 0.5271 - val_f1: 0.0000e+00 - val_recall: 0.0000e+00 - val_precision: 0.0000e+00
Epoch 68/100
 - 0s - loss: -6.8648e-01 - binary_accuracy: 0.4932 - f1: 0.0000e+00 - recall: 0.0000e+00 - precision: 0.0000e+00 - val_loss: -4.6883e-01 - val_binary_accuracy: 0.5271 - val_f1: 0.0000e+00 - val_recall: 0.0000e+00 - val_precision: 0.0000e+00
Epoch 69/100
 - 0s - loss: -6.8094e-01 - binary_accuracy: 0.4932 - f1: 0.0000e+00 - recall: 0.0000e+00 - precision: 0.0000e+00 - val_loss: -4.6649e-01 - val_binary_accuracy: 0.5271 - val_f1: 0.0000e+00 - val_recall: 0.0000e+00 - val_precision: 0.0000e+00
Epoch 70/100
 - 0s - loss: -7.0795e-01 - binary_accuracy: 0.4932 - f1: 0.0000e+00 - recall: 0.0000e+00 - precision: 0.0000e+00 - val_loss: -4.8995e-01 - val_binary_accuracy: 0.5271 - val_f1: 0.0000e+00 - val_recall: 0.0000e+00 - val_precision: 0.0

 - 0s - loss: -7.4783e-01 - binary_accuracy: 0.4936 - f1: 0.0088 - recall: 0.0045 - precision: 0.3649 - val_loss: -5.3953e-01 - val_binary_accuracy: 0.5256 - val_f1: 0.0119 - val_recall: 0.0061 - val_precision: 0.2431


C:\ProgramData\Anaconda3\envs\GCNFTG\lib\site-packages\ipykernel_launcher.py:38: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index


ValueError: multi_class must be in ('ovo', 'ovr')